### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1d91a31445b510471080de5f8b8d04a58ae2f5247c748a8d3f0f49d31db98b4e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

Now we import some of the libraries usually needed by our workload.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Preprocessing

In this Colab, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [4]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

For convenience, given that the dataset is small, we first construct a Pandas dataframe, tune the schema, and then convert it into a Spark dataframe.

In [6]:
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [7]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

### Your task

If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [8]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


In [24]:
# Initialize the KMeans model
kmeans = KMeans().setK(2).setSeed(1)  # setK(2) sets number of clusters to 2
model = kmeans.fit(features)  # Train the model


In [10]:
predictions = model.transform(features)


In [11]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print(f"Silhouette with squared euclidean distance = {silhouette}")

Silhouette with squared euclidean distance = 0.8342904262826145


Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

*HINT*: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two series.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [12]:
import numpy as np
import pandas as pd

predicted_clusters = predictions.select("prediction").toPandas()['prediction']
actual_labels = np.array(labels)


In [13]:
direct_match_count = np.count_nonzero(predicted_clusters == actual_labels)

swapped_match_count = np.count_nonzero(predicted_clusters != actual_labels)


In [16]:
correctly_clustered = __builtins__.max(direct_match_count, swapped_match_count)
print(f"Number of data points clustered correctly: {correctly_clustered}")


Number of data points clustered correctly: 486


Now perform dimensionality reduction on the ```features``` using [SVD](https://spark.apache.org/docs/latest/mllib-dimensionality-reduction), available as well in MLlib.

Reduce the dimensionality to **2**, effectively reducing the dataset size of a **15X** factor. Name the new dataset as ```svdFeatures```

In [18]:
from pyspark.ml.feature import PCA, VectorAssembler


In [19]:
vector_assembler = VectorAssembler(inputCols=features.columns, outputCol = "assembled_features")
features_vectorized = vector_assembler.transform(features)

In [20]:
pca = PCA(inputCol = "assembled_features", outputCol="svdfeatures", k=2)

model = pca.fit(features_vectorized)
result = model.transform(features_vectorized)
svdFeatures = result.select("svdFeatures")

In [25]:
from pyspark.ml.clustering import KMeans

kmeans_svd = KMeans().setK(2).setSeed(1).setFeaturesCol("svdFeatures")
model_svd = kmeans_svd.fit(svdFeatures)

predictions_svd = model_svd.transform(svdFeatures)

In [28]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator_svd = ClusteringEvaluator(featuresCol = 'svdFeatures')
silhouette_svd = evaluator_svd.evaluate(predictions_svd)
print(f"Silhouette with squared euclidean distance (SVD features) = {silhouette_svd}")

Silhouette with squared euclidean distance (SVD features) = 0.8348610363444832


Now run K-means with the same parameters as above, but on the ```svdFeatures``` produced by the SVD reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [29]:
predicted_clusters_svd = predictions_svd.select("prediction").toPandas()['prediction']

In [30]:
direct_match_count_svd = np.count_nonzero(predicted_clusters_svd == actual_labels)

In [31]:
swapped_match_count_svd = np.count_nonzero(predicted_clusters_svd != actual_labels)

In [36]:
correctly_clustered_svd = __builtins__.max(direct_match_count_svd, swapped_match_count_svd)
print(f"Number of data points clustered correctly (SVD features): {correctly_clustered_svd}")

Number of data points clustered correctly (SVD features): 486


#### **Submission Intruction:**

#### Click File -> Download -> Download .ipynb, and upload the downloaded file to Blackboard.